In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.utils import np_utils, print_summary
import pandas as pd
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
 
# Image dimensions
img_width, img_height = 150, 150 
 


C:\Users\chall\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.


In [9]:
def create_model(p, input_shape=(32, 32, 3)):
    # Initialising the CNN
    model = Sequential()
    # Convolution + Pooling Layer 
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Convolution + Pooling Layer 
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Convolution  followed by Pooling Layer 
    # increasing the number of filters as we have pooled out features reduced features. we can afford to apply 64 filters.
    
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Convolution + Pooling Layer 
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Flattening
    model.add(Flatten())
    # Fully connection ANN
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(p))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(p/2))
    model.add(Dense(6, activation='sigmoid'))
        # Compiling the CNN
    optimizer = Adam(lr=1e-3)
    metrics=['accuracy']
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    filepath = "cnn_v0.h5"
    checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    #checkpoint2 = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    callbacks_list = [checkpoint1]
    return model

In [10]:
def run_training(bs=32, epochs=10):
    
    train_datagen = ImageDataGenerator(rescale = 1./255, 
                                       shear_range = 0.2, 
                                       zoom_range = 0.2, 
                                       horizontal_flip = True)
    test_datagen = ImageDataGenerator(rescale = 1./255)
 
    training_set = train_datagen.flow_from_directory('C:/Users/chall/Desktop/dataset-train',
                                                 target_size = (img_width, img_height),
                                                 batch_size = bs,
                                                 class_mode = "categorical"
                                                    )
                                                 
    test_set = test_datagen.flow_from_directory('C:/Users/chall/Desktop/dataset-test',
                                            target_size = (img_width, img_height),
                                            batch_size = bs,
                                            class_mode = 'categorical')
                                            
    model = create_model(p=0.6, input_shape=(img_width, img_height, 3))                                  
    model.fit_generator(training_set,
                         steps_per_epoch=1000/bs,
                         epochs = epochs,
                         validation_data = test_set,
                         validation_steps = 2000/bs
                    )
    return model


In [12]:
classifier = run_training(bs=32, epochs=10)#100 epochs

Found 1915 images belonging to 6 classes.
Found 612 images belonging to 6 classes.
Epoch 1/10
32/31 [==============================] - 67s 2s/step - loss: 0.5268 - acc: 0.7848 - val_loss: 0.4487 - val_acc: 0.8333
Epoch 2/10
32/31 [==============================] - 63s 2s/step - loss: 0.4688 - acc: 0.8301 - val_loss: 0.4544 - val_acc: 0.8333
Epoch 3/10
32/31 [==============================] - 61s 2s/step - loss: 0.4597 - acc: 0.8332 - val_loss: 0.4483 - val_acc: 0.8333
Epoch 4/10
32/31 [==============================] - 63s 2s/step - loss: 0.4536 - acc: 0.8330 - val_loss: 0.4561 - val_acc: 0.8333
Epoch 5/10
32/31 [==============================] - 62s 2s/step - loss: 0.4571 - acc: 0.8333 - val_loss: 0.4545 - val_acc: 0.8333
Epoch 6/10
32/31 [==============================] - 63s 2s/step - loss: 0.4497 - acc: 0.8333 - val_loss: 0.4476 - val_acc: 0.8333
Epoch 7/10
32/31 [==============================] - 61s 2s/step - loss: 0.4475 - acc: 0.8333 - val_loss: 0.4594 - val_acc: 0.8333
Epoch 8

In [13]:
classifier.save('garbage1.h5')

In [14]:
classifier.save('garbage')

In [15]:
import cv2
from keras.models import load_model
import numpy as np
from collections import deque

model1 = load_model('garbage')